In [1]:
import pandas as pd

In [2]:
import psycopg2
import socket

# information used to create a database connection
sqluser = 'cades' if socket.gethostname() == 'osl-va' else 'sudarshan'
dbname = 'mimic'
schema_name = 'mimiciii'

# Connect to postgres with a copy of the MIMIC-III database
con = psycopg2.connect(dbname=dbname, user=sqluser, host="/var/run/postgresql")

In [3]:
cats = pd.read_csv('cats.csv')
max_limit = 5

queries = []
for category, n_notes in zip(cats['category'], cats['number_of_notes']):
    limit = min(max_limit, n_notes) if max_limit > 0 else n_notes
    if limit == max_limit:
        q = f"""
        select * from correctnotes where category=\'{category}\' order by random() limit {limit};
        """
    else:
        q = f"""
        select * from correctnotes where category=\'{category}\';
        """
    queries.append(q)

dfs = []
for q in queries:
    df = pd.read_sql_query(q, con)
    dfs.append(df)
    
df = pd.concat(dfs)
df.set_index('row_id', inplace=True)
df.shape

(70, 9)

In [1]:
from QuickUMLS.quickumls import QuickUMLS

In [2]:
umlspath = '/storage/UMLS-Storage/quickumls'

In [3]:
text = "The ulna has dislocated posteriorly from the trochlea of the humerus."

In [4]:
matcher = QuickUMLS(umlspath)

In [5]:
matcher.match(text, best_match=True, ignore_syntax=False)

[[{'start': 61,
   'end': 68,
   'ngram': 'humerus',
   'term': 'humerus',
   'cui': 'C0020164',
   'similarity': 1.0,
   'semtypes': {'T023'},
   'preferred': 1}],
 [{'start': 4,
   'end': 8,
   'ngram': 'ulna',
   'term': 'ulna',
   'cui': 'C0041600',
   'similarity': 1.0,
   'semtypes': {'T023'},
   'preferred': 1}],
 [{'start': 13,
   'end': 23,
   'ngram': 'dislocated',
   'term': 'Dislocated',
   'cui': 'C0012691',
   'similarity': 0.7777777777777778,
   'semtypes': {'T037'},
   'preferred': 1}]]